!!! Здесь описана очень старая версия бустинга. Текущая реализация для [мультибида](https://bitbucket.org/playgendary-dev/gap-runner/src/master/) и для [дефолтов](https://bitbucket.org/playgendary-dev/geo-defaults/src/master/).

В схеме прогнозирования LTV для закупки трафика есть два этапа:

- бустинг для прогнозирования монетизации ранних дней: для рекламы RPI7, для инапов RPI7, для подписок конверсия с 0 по 6 день;
- полученные из бустинга прогнозы умножаются на соответствующие коэффициенты, чтобы получить прогноз LTV.

На основании прогноза LTV и желаемой маржинальности выставляются биды (цена, за которую мы готовы купить пользователя).

<a href="https://playgendary.atlassian.net/wiki/spaces/analytics/pages/57770079">Описание биддинга</a>

<a href="https://playgendary.atlassian.net/wiki/spaces/analytics/pages/55574630">Биды для стран по умолчанию и мультибиддинг</a>

### 1. Выгрузка данных

In [1]:
import pandas as pd
import numpy as np

import pandas_gbq
import pydata_google_auth
import logging

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform'
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    auth_local_webserver=True,
)

logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

def execute(sql):
    res = pandas_gbq.read_gbq(
        sql,
        project_id='playgendary-bi',
        credentials=credentials,
    )
    
    return res

Минимальный разрез, на который мы выставляем бид - тайтл + страна + паблишер. Также добавим дополнительные фичи: название игры (общее для обеих платформ), платформа и версия приложения. Количество инсталлов используем в качестве весов. Не забываем про три разных таргета (ad RPI7, inapp RPI7 и subs conv6). Исходя из этого выгружаем данные для обучения (train) и теста (test).

Небольшие пояснения по запросу:
- except_camp - исключаем оптимизационные кампании, так как их метрики обычно выше и для них биды выставляются отдельно
- в запросе препроцессим названия паблишеров (siteId)
- исключаем из источников органику и тестовые

In [2]:
target_day = 7
subs_day = 6

In [3]:
query = '''
  WITH
  except_camp AS (
  SELECT
    DISTINCT campaignName AS campaignName
  FROM
    `playgendary-bi.aggregated_data.daily_stat`
  WHERE
    date = "{date}"
    AND ((mediaSource = "unityads_int"
        AND (REGEXP_CONTAINS(LOWER(campaignName), "7dreten")
          OR REGEXP_CONTAINS(LOWER(campaignName), "roas")))
      OR (mediaSource = "googleadwords_int"
        AND REGEXP_CONTAINS(LOWER(campaignName), "action"))
      OR (mediaSource = "Facebook Ads"
        AND (REGEXP_CONTAINS(LOWER(campaignName), "_vo_")
          OR REGEXP_CONTAINS(LOWER(campaignName), "_aeo_"))))),
  installs_t AS (
  SELECT
    LOWER(apps.app_name) AS appName,
    LOWER(inst.appId) AS appId,
    LOWER(inst.mediaSource) AS mediaSource,
    LOWER(CONCAT(versions.app_version, '-', versions.appId)) AS appVersion,
    CASE
      WHEN inst.mediaSource = "ironsource_int" THEN LOWER(REGEXP_EXTRACT(inst.siteId, r"[a-z\.]*_|^(\d+)_?\w*"))
      WHEN inst.mediaSource = "tapjoy_int" THEN LOWER(REGEXP_EXTRACT(inst.siteId, r".+_(\w*)"))
      WHEN inst.mediaSource = "vungle_int" THEN LOWER(REGEXP_EXTRACT(inst.siteId, r"^([^_]+)"))
      WHEN inst.mediaSource = "googleadwords_int" THEN LOWER(
    IF
      (inst.siteId = "null" OR inst.siteId is null,
        "GDN",
        inst.siteId))
      WHEN inst.mediaSource IN ("liftoff_int", "Apple Search Ads") THEN LOWER(inst.mediaSource)
      WHEN inst.mediaSource IN ("Facebook Ads",
      "snapchat_int") THEN LOWER(adsetId)
    ELSE
    LOWER(inst.siteId)
  END
    AS siteId,
    LOWER(inst.platform) AS platform,
    LOWER(inst.countryCode) AS countryCode,
    date AS installDate,
    SUM(installs) AS installs
  FROM
    `playgendary-bi.aggregated_data.daily_stat` inst
  INNER JOIN
    `marketing-analytics-235713.auxiliary.apps_dictionary` AS apps
  ON
    apps.app_id = inst.appId
  INNER JOIN
    `marketing-analytics-235713.auxiliary.app_versions` AS versions
  ON
    versions.appId = inst.appId
    AND versions.day = inst.date
  WHERE
    date = "{date}"
    AND LOWER(inst.mediaSource) NOT IN ('organic', 'appsflyer_test', 'appsflyer_sdk_test_int')
    AND inst.campaignName NOT IN (
    SELECT
      campaignName
    FROM
      except_camp)
  GROUP BY
    appName,
    appId,
    appVersion,
    mediaSource,
    siteId,
    platform,
    countryCode,
    installDate),
  subs AS (
  SELECT
    LOWER(appId) AS appId,
    LOWER(mediaSource) AS mediaSource,
    CASE
      WHEN mediaSource = "ironsource_int" THEN LOWER(REGEXP_EXTRACT(siteId, r"[a-z\.]*_|^(\d+)_?\w*"))
      WHEN mediaSource = "tapjoy_int" THEN LOWER(REGEXP_EXTRACT(siteId, r".+_(\w*)"))
      WHEN mediaSource = "vungle_int" THEN LOWER(REGEXP_EXTRACT(siteId, r"^([^_]+)"))
      WHEN mediaSource = "googleadwords_int" THEN LOWER(
    IF
      (siteId = "null" OR siteId is null,
        "GDN",
        siteId))
      WHEN mediaSource IN ("liftoff_int", "Apple Search Ads") THEN LOWER(mediaSource)
      WHEN mediaSource IN ("Facebook Ads",
      "snapchat_int") THEN LOWER(adsetId)
    ELSE
    LOWER(siteId)
  END
    AS siteId,
    LOWER(countryCode) AS countryCode,
    installDate AS installDate,
    SUM(eventCount) AS subs_conv
  FROM
    `playgendary-bi.aggregated_data.daily_inapp_revenue`
  WHERE
    installDate = '{date}'
    AND eventDate BETWEEN installDate
    AND DATE_ADD(installDate, INTERVAL {subs_day} DAY)
    AND eventName NOT LIKE '%trial%'
    AND eventName LIKE '%subscription_server%'
    AND revenue IS NOT NULL
    AND revenue != 0.0
    AND LOWER(mediaSource) NOT IN ('organic', 'appsflyer_test', 'appsflyer_sdk_test_int')
    AND campaignName NOT IN (
    SELECT
      campaignName
    FROM
      except_camp)
  GROUP BY
    appId,
    mediaSource,
    siteId,
    countryCode,
    installDate ),
  ads AS (
  SELECT
    LOWER(agg.appId) AS appId,
    LOWER(agg.mediaSource) AS mediaSource,
    CASE
      WHEN agg.mediaSource = "ironsource_int" THEN LOWER(REGEXP_EXTRACT(agg.siteId, r"[a-z\.]*_|^(\d+)_?\w*"))
      WHEN agg.mediaSource = "tapjoy_int" THEN LOWER(REGEXP_EXTRACT(agg.siteId, r".+_(\w*)"))
      WHEN agg.mediaSource = "vungle_int" THEN LOWER(REGEXP_EXTRACT(agg.siteId, r"^([^_]+)"))
      WHEN agg.mediaSource = "googleadwords_int" THEN LOWER(
    IF
      (agg.siteId = "null",
        "GDN",
        agg.siteId))
      WHEN agg.mediaSource IN ("liftoff_int", "Apple Search Ads") THEN LOWER(agg.mediaSource)
      WHEN agg.mediaSource IN ("Facebook Ads",
      "snapchat_int") THEN LOWER(adsetId)
    ELSE
    LOWER(agg.siteId)
  END
    AS siteId,
    LOWER(agg.platform) AS platform,
    LOWER(agg.countryCode) AS countryCode,
    agg.installDate AS installDate,
    SUM(
    IF
      (cohortDay <= {target_day},
        adRevenue,
        0)) AS ad_revenue,
    SUM(
    IF
      (cohortDay <= {target_day},
        inappRevenue,
        0)) AS inapp_revenue
  FROM
        `playgendary-bi.aggregated_data.aggregated_cohort_revenue` AS agg
  WHERE
    agg.installDate = '{date}'
    AND LOWER(agg.mediaSource) NOT IN ('organic', 'appsflyer_test', 'appsflyer_sdk_test_int')
    AND agg.campaignName NOT IN (
    SELECT
      campaignName
    FROM
      except_camp)
  GROUP BY
    appId,
    mediaSource,
    installDate,
    siteId,
    platform,
    countryCode )
SELECT
  inst.appName,
  inst.appId,
  inst.appVersion,
  inst.mediaSource,
  inst.siteId,
  inst.platform,
  inst.countryCode,
  inst.installDate,
  IFNULL(ad_revenue,
    0) AS ad_revenue,
  IFNULL(inapp_revenue,
    0) AS purch_revenue,
  IFNULL(subs_conv,
    0) AS subs_conv,
  installs
FROM
  installs_t AS inst
LEFT JOIN
  ads
ON
  ads.installDate = inst.installDate
  AND ads.appId = inst.appId
  AND ads.mediaSource = inst.mediaSource
  AND ads.siteId = inst.siteId
  AND ads.platform = inst.platform
  AND ads.countryCode = inst.countryCode
LEFT JOIN
  subs
ON
  subs.installDate = inst.installDate
  AND subs.appId = inst.appId
  AND subs.mediaSource = inst.mediaSource
  AND subs.siteId = inst.siteId
  AND subs.countryCode = inst.countryCode
WHERE
  installs > 0
'''

Выбираем даты для train и test. В данный момент для обучения используется 7 дней истории. Для теста также возьмем 7 дней. Чтобы приблизить ситуацию к боевой, не забываем, что между последней датой из train и первой из test должно быть 7+3 дней (7 - для накопления дней факта и 3 из-за задержки в данных).

In [4]:
train_dates = pd.date_range('2019-11-15', '2019-11-21').strftime('%Y-%m-%d').tolist()
test_dates = pd.date_range('2019-12-01', '2019-12-07').strftime('%Y-%m-%d').tolist()

In [5]:
for date in train_dates:
    df = execute(query.format(date=date, target_day=target_day, subs_day=subs_day))
    df.to_parquet('train/'+date+'.pq', engine='pyarrow', index=False)

Requesting query... 
Query running...
Job ID: c6268871-d110-4333-ab83-6b98473b30de
  Elapsed 6.66 s. Waiting...
  Elapsed 8.78 s. Waiting...
  Elapsed 10.64 s. Waiting...
Query done.
Processed: 209.6 MB Billed: 210.0 MB
Standard price: $0.00 USD

Got 285086 rows.

Total time taken 47.93 s.
Finished at 2019-12-17 15:29:09.
Requesting query... 
Query running...
Job ID: 4a02646e-2296-4d67-9255-f529f5a7a6c2
  Elapsed 6.22 s. Waiting...
  Elapsed 7.29 s. Waiting...
  Elapsed 9.38 s. Waiting...
  Elapsed 11.39 s. Waiting...
Query done.
Processed: 249.8 MB Billed: 250.0 MB
Standard price: $0.00 USD

Got 336801 rows.

Total time taken 55.65 s.
Finished at 2019-12-17 15:30:05.
Requesting query... 
Query running...
Job ID: c9197dcf-8d8c-4d85-9a57-35b9cc88d3f3
  Elapsed 7.79 s. Waiting...
  Elapsed 9.33 s. Waiting...
  Elapsed 11.36 s. Waiting...
Query done.
Processed: 246.6 MB Billed: 247.0 MB
Standard price: $0.00 USD

Got 328326 rows.

Total time taken 53.95 s.
Finished at 2019-12-17 15:31:00.

In [6]:
for date in test_dates:
    df = execute(query.format(date=date, target_day=target_day, subs_day=subs_day))
    df.to_parquet('test/'+date+'.pq', engine='pyarrow', index=False)

Requesting query... 
Query running...
Job ID: a11594a3-4ac3-476c-be36-0841b6a46185
  Elapsed 7.78 s. Waiting...
Query done.
Processed: 193.7 MB Billed: 194.0 MB
Standard price: $0.00 USD

Got 310045 rows.

Total time taken 48.61 s.
Finished at 2019-12-17 15:34:45.
Requesting query... 
Query running...
Job ID: 825f5007-0119-4a37-b278-320b23f3baee
  Elapsed 6.23 s. Waiting...
Query done.
Processed: 146.6 MB Billed: 147.0 MB
Standard price: $0.00 USD

Got 238771 rows.

Total time taken 37.84 s.
Finished at 2019-12-17 15:35:24.
Requesting query... 
Query running...
Job ID: 24fc2fc9-6f55-4d53-a29e-bf233097f7f1
Query done.
Processed: 139.5 MB Billed: 140.0 MB
Standard price: $0.00 USD

Got 238299 rows.

Total time taken 39.09 s.
Finished at 2019-12-17 15:36:03.
Requesting query... 
Query running...
Job ID: 0ba77d90-0866-4379-ba61-d324f92f4359
  Elapsed 7.77 s. Waiting...
Query done.
Processed: 138.3 MB Billed: 139.0 MB
Standard price: $0.00 USD

Got 243478 rows.

Total time taken 41.52 s.
Fi

In [7]:
train = pd.read_parquet('train/')
test = pd.read_parquet('test/')

In [8]:
train.sample(5)

,appName,appId,appVersion,mediaSource,siteId,platform,countryCode,installDate,ad_revenue,purch_revenue,subs_conv,installs
615206,polysphere,com.playgendary.polyspherecoolgame,1.4.6-com.playgendary.polyspherecoolgame,unityads_int,1624308,android,ke,2019-11-16,0.02201,0.0,0,1
1797897,tomb of the mask: color,com.playgendary.tombpaint,1.0.3-com.playgendary.tombpaint,ironsource_int,262361,android,hn,2019-11-21,0.01340,0.0,0,1
960717,tomb of the mask,id1057889290,1.7.2-id1057889290,ironsource_int,237563,ios,es,2019-11-18,0.00002,0.0,0,1
1233735,polysphere,id1440756080,1.4.6-id1440756080,applovin_int,b613f60ec8caf47e18b0e2dd26d537a4,ios,us,2019-11-19,2.46159,0.0,0,5
174883,tomb of the mask,com.playgendary.tom,1.4.1-com.playgendary.tom,unityads_int,3121203,android,in,2019-11-15,0.04828,0.0,0,1


### 2. Подготовка test и label encoder

In [9]:
features = [
    'appName',
    'appId',
    'appVersion',
    'mediaSource',
    'siteId',
    'platform',
    'countryCode'
]

weight_col = 'installs'
target_cols = ['ad_revenue', 'purch_revenue', 'subs_conv']

Сгруппируем данные, так как мы не используем информацию по installDate

In [10]:
train_grp = train.groupby(features, as_index=False)[[weight_col] + target_cols].sum()
test_grp = test.groupby(features, as_index=False)[[weight_col] + target_cols].sum()

Для теста используем последнюю версию приложения c достаточным количеством инсталлов

In [11]:
MIN_APPVERSION_INSTALLS = 5000

def select_appVersion(df):
    if df.shape[0] == 1:
        return df['appVersion'].values[0]
        
    df['enough'] = (df['installs'] >= MIN_APPVERSION_INSTALLS)
    if df['enough'].sum() > 1:
        return df.loc[df['enough']]['appVersion'].max()
    else:
        return df.nlargest(1, 'installs')['appVersion'].values[0]

In [12]:
test_grp.drop(['appVersion'], axis=1, inplace=True)
tmp = train_grp.groupby(['appId', 'appVersion'])['installs'].sum().reset_index()
versions = tmp.groupby('appId').apply(select_appVersion)\
              .to_frame(name='appVersion').reset_index()
test_grp = test_grp.merge(versions, on='appId', how='left')

Отбираем только тех паблишеров, которые были в обучении с достаточным количеством инсталлов

In [13]:
MIN_INSTALLS = 100

test_filtered = test_grp.merge(train_grp, on=features, suffixes=("", "_train"))
test_filtered = test_filtered.loc[test_filtered[weight_col + "_train"] >= MIN_INSTALLS].copy()

Далее используем label encoder https://bitbucket.org/playgendary-dev/label-encoder/src/master/

In [14]:
from labenc import LabelEncoder

In [15]:
encoder = LabelEncoder().fit(train_grp[features])
encoder.save('encoder.pkl')

train_encoded = encoder.transform(train_grp)
test_encoded = encoder.transform(test_filtered)

### 3. Подбор гиперпараметров

In [16]:
import lightgbm as lgb

from hyperopt import hp, tpe, space_eval, Trials
from hyperopt.fmin import fmin
from hyperopt.pyll.base import scope
@scope.define
def to_int(x):
    return int(max(1, x))

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


Помимо гиперпараметров самого бустинга, добавим еще weight_power - степень, в которую возводим количество инсталлов (веса), чтобы уменьшить влияние крупных когорт. В качестве оптимизационной метрики для рекламного дохода используем MSE.

In [17]:
target_col = 'ad_revenue'

In [18]:
space = {
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
    'num_leaves': hp.choice('num_leaves', [2**5 - 1, 2**6 - 1, 2**7 - 1]),
    
    'max_cat_threshold': scope.to_int(hp.quniform('max_cat_threshold', 2, 500, 15)),
    'cat_l2': scope.to_int(hp.quniform('cat_l2', 1, 1000, 25)),
    'cat_smooth': scope.to_int(hp.quniform('cat_smooth', 2, 1000, 25)),
    
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, 1.),
    'bagging_freq': scope.to_int(hp.quniform('bagging_freq', 0, 3, 1)),
    'feature_fraction': hp.uniform('feature_fraction', 0.4, 1.),
    
    'max_bin': hp.choice('max_bin', [2**7 - 1, 2**8 - 1, 2**9-1, 2**10-1]),
    
    'boosting': 'gbdt',
    'reg_sqrt': False,
    
    'min_data_per_group': 1,
    'min_data_in_leaf': 1,
    'min_sum_hessian_in_leaf': hp.uniform('min_sum_hessian_in_leaf', 1e-4, 1),
    
    
    "weight_power": hp.uniform('weight_power', 0.1, 1.), 
    
    'verbose': 1,
    'objective': 'mse',
    'metric': 'mse',
    'bagging_seed': 322,
    'feature_fraction_seed': 322,
    'first_metric_only': True,
}

In [19]:
aux_params = {
    'num_boost_round': 500,
    'early_stopping_rounds': 5,
    'verbose_eval': 50
}

In [20]:
def objective(params):
    
    lgb_train = lgb.Dataset(
        data=train_encoded[features],
        label=train_encoded[target_col] / train_encoded[weight_col],
        weight=train_encoded[weight_col] ** (params['weight_power']),
        categorical_feature=features)
    params.pop('weight_power')
    
    lgb_test = lgb.Dataset(
        data=test_encoded[features],
        label=test_encoded[target_col] / test_encoded[weight_col],
        weight=test_encoded[weight_col],
        categorical_feature=features)
    
    res = lgb.train(
        params=params,
        train_set=lgb_train,
        valid_sets=lgb_test,
        **aux_params)
    
    return res.best_score['valid_0']['l2']

In [21]:
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')



Training until validation scores don't improve for 5 rounds.
  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')



[50]	valid_0's l2: 0.000406019                      
Early stopping, best iteration is:                  
[89]	valid_0's l2: 0.000341601
Training until validation scores don't improve for 5 rounds.                     
[50]	valid_0's l2: 0.000721573                                                   
[100]	valid_0's l2: 0.000512274                                                  
Early stopping, best iteration is:                                               
[121]	valid_0's l2: 0.000480128
Training until validation scores don't improve for 5 rounds.                     
[50]	valid_0's l2: 0.000713697                                                   
[100]	valid_0's l2: 0.000508399                                                  
Early stopping, best iteration is:                                               
[96]	valid_0's l2: 0.000504479
Training until validation scores don't improve for 5 rounds.                     
Early stopping, best iteration is:                            

In [22]:
best_params = space_eval(space, best)
best_params

{'bagging_fraction': 0.5792207964941026,
 'bagging_freq': 1,
 'bagging_seed': 322,
 'boosting': 'gbdt',
 'cat_l2': 900,
 'cat_smooth': 100,
 'feature_fraction': 0.5931811855652239,
 'feature_fraction_seed': 322,
 'first_metric_only': True,
 'learning_rate': 0.12964016071429868,
 'max_bin': 511,
 'max_cat_threshold': 495,
 'metric': 'mse',
 'min_data_in_leaf': 1,
 'min_data_per_group': 1,
 'min_sum_hessian_in_leaf': 0.9074814514558993,
 'num_leaves': 127,
 'objective': 'mse',
 'reg_sqrt': False,
 'verbose': 1,
 'weight_power': 0.8009176625701054}

Повторяем процедуру для других таргетов. Для инапов используем MSE, для подписок - cross_entropy (xentropy в валидации).

### 4. Обучение

In [25]:
lgb_train = lgb.Dataset(
    data=train_encoded[features],
    label=train_encoded[target_col] / train_encoded[weight_col],
    weight=train_encoded[weight_col] ** (best_params['weight_power']),
    categorical_feature=features)

lgb_test = lgb.Dataset(
    data=test_encoded[features],
    label=test_encoded[target_col] / test_encoded[weight_col],
    weight=test_encoded[weight_col],
    categorical_feature=features)

res = lgb.train(
    params=best_params,
    train_set=lgb_train,
    valid_sets=lgb_test,
    **aux_params)

test_encoded[f'{target_col}_pred'] = res.predict(test_encoded[features])

Training until validation scores don't improve for 5 rounds.
[50]	valid_0's l2: 0.000267916
Early stopping, best iteration is:
[92]	valid_0's l2: 0.000220275


Повторяем для других таргетов.

### 5. Меряем ошибку

In [27]:
from sklearn.metrics import  mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

def cross_entropy(y, p, w):
    return np.sum((-y*np.log(p)-(1-y)*np.log(1-p))*w)/np.sum(w)

def calc_metrics(preds, y, sample_weight, xentropy=False):
    output = {
        'weighted_cross_entropy': cross_entropy(y, preds, sample_weight) if xentropy else None,
        'weighted_mean_squared_error': mean_squared_error(y, preds, sample_weight),
        'weighted_mean_absolute_error': mean_absolute_error(y, preds, sample_weight),
        'mean_squared_error': mean_squared_error(y, preds),
        'mean_absolute_error': mean_absolute_error(y, preds),
        'median_absolute_error': median_absolute_error(y, preds),
        'weighted_r2_score': r2_score(y, preds, sample_weight),
        'r2_score': r2_score(y, preds),

        'target_test_mean': y.mean(),
        'prediction_test_mean': preds.mean(),

        'weighted_target_test_mean': np.average(y, weights=sample_weight),
        'weighted_prediction_test_mean': np.average(preds, weights=sample_weight),
    }
    return output

In [28]:
calc_metrics(test_encoded[f'{target_col}_pred'], 
             test_encoded[target_col]/test_encoded[weight_col], 
             test_encoded[weight_col])

{'weighted_cross_entropy': None,
 'weighted_mean_squared_error': 0.00022027479774115963,
 'weighted_mean_absolute_error': 0.005472188882348486,
 'mean_squared_error': 0.0018423297231395304,
 'mean_absolute_error': 0.01603167190046626,
 'median_absolute_error': 0.00453604301622737,
 'weighted_r2_score': 0.9433237602636073,
 'r2_score': 0.780700028234995,
 'target_test_mean': 0.05042287309610533,
 'prediction_test_mean': 0.05198970049395405,
 'weighted_target_test_mean': 0.027948535274660295,
 'weighted_prediction_test_mean': 0.02880063109541742}

Репозиторий https://bitbucket.org/playgendary-dev/gap-runner/src/master/

Брифинг по бустингу https://drive.google.com/drive/folders/1KmqCF-x_4f7fDeDEgyImlTlpIsoXPFiv